In [24]:
import os  
from selenium import webdriver  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options  
import unittest
import multiprocessing as mp
from multiprocessing import Pool
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException,TimeoutException,WebDriverException

import time, csv
import datetime

import zipfile
import glob

from zipfile import ZipFile

In [25]:
BSEBCURL='https://www.bseindia.com/markets/MarketInfo/BhavCopy.aspx'
BSEDELI = 'https://www.bseindia.com/markets/equity/EQReports/GrossShortPos.aspx?flag=0'

In [26]:
def setWebDriver(Dest_Data_Folder,url):
    chrome_options = Options()
    prefs = {"download.default_directory": Dest_Data_Folder, "download.prompt_for_download": False,}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
    # Changing the chrome downloads from default directory to Destination directory
    params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath':Dest_Data_Folder}}
    driver.execute("send_command", params)
    driver.delete_all_cookies()
    driver.get(url) 
    return driver

In [27]:
def downloadBhavCopy():
    # names in the given directory 
    listOfFile = os.listdir('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies')
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies', entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isfile(fullPath):
            os.remove(fullPath)
    
    #Initializing driver
    driver=setWebDriver('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies', BSEBCURL)
    x = datetime.datetime.now()

    month = x.strftime("%b")
    year = x.strftime("%Y")
    date = x.strftime("%d")
    date = int(date) - 1
#     print(date)

    select = Select(driver.find_element_by_id('ContentPlaceHolder1_fdate1'))
    select.select_by_visible_text(str(date))
    time.sleep(2)

    select = Select(driver.find_element_by_id('ContentPlaceHolder1_fmonth1'))
    select.select_by_visible_text(month)
    time.sleep(2)

    select = Select(driver.find_element_by_id('ContentPlaceHolder1_fyear1'))
    select.select_by_visible_text(year)
    time.sleep(2)

    driver.find_element_by_id('ContentPlaceHolder1_btnSubmit').click()
    time.sleep(3)

    driver.find_element_by_id('ContentPlaceHolder1_btnHylSearBhav').click()
    time.sleep(3)
    
    
    
    listOfFile = os.listdir('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies')
    allFiles = list()
    # Iterate over all the entries
    global fileName
    for entry in listOfFile:
#         if '.ZIP' not in entry:
            fileName = entry
            print(fileName)
    
    
    zf = ZipFile('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies\\' + fileName, 'r')
    zf.extractall('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies')
    zf.close()
    
    time.sleep(10)
    driver.close()

In [28]:
downloadBhavCopy()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


 
EQ110221_CSV.ZIP


In [29]:
def downloadDeliverables():
    # names in the given directory 
    listOfFile = os.listdir('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables')
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables', entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isfile(fullPath):
            os.remove(fullPath)
    
    #Initializing driver
    driver=setWebDriver('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables', BSEDELI)
    driver.find_element_by_id('ContentPlaceHolder1_hylinksearch').click()
    time.sleep(3)
    
    
    listOfFile = os.listdir('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables')
    allFiles = list()
    # Iterate over all the entries
    global deliName
    global n
    for entry in listOfFile:
#         if '.ZIP' not in entry:
            deliName = entry
            print(deliName)
    
    from zipfile import ZipFile
    zf = ZipFile('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables\\' + deliName, 'r')
    zf.extractall('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables')
    zf.close()
    
    time.sleep(10)
    driver.close()
# df = pd.read_csv("D:\MSIT\Second Year\Data Science\Project Trial\SCBSEALL1202.TXT",delimiter='|')
# df.to_csv('D:\MSIT\Second Year\Data Science\Project Trial\SCBSEALL1202.CSV', index = False)
    n = deliName.replace(".zip","")
    df = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables\\" + n + ".TXT", delimiter='|')
    df.to_csv('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables\\' + n + ".CSV", index = False)

In [30]:
downloadDeliverables()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


SCBSEALL1102.zip


In [31]:
def convertBhavToStock(df):
    name = fileName.replace("_CSV.ZIP", ".CSV")
#     print(name)
    bhav = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\BhavCopies\\" + name)
    deliverableData = pd.read_csv('C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Deliverables\\' + n + ".CSV")
    deliverableData = deliverableData.drop(columns=["DATE", "DELIVERY VAL", "DAY'S VOLUME", "DAY'S TURNOVER"])
    
    deliverableData.rename(columns={"SCRIP CODE": "SC_CODE", "DELIVERY QTY":"Deliverable Quantity", "DELV. PER.":"% Deli. Qty to Traded Qty"}, inplace = True)
    print(deliverableData)
    df["SC_CODE"] = bhav["SC_CODE"]
    df["SC_NAME"] = bhav["SC_NAME"]
    df["Open Price"] = bhav["OPEN"]
    df["High Price"] = bhav["HIGH"]
    df["Low Price"] = bhav["LOW"]
    df["Close Price"] = bhav["CLOSE"]
    df["WAP"] = bhav["NET_TURNOV"] / bhav["NO_OF_SHRS"]
    df["No.of Shares"] = bhav["NO_OF_SHRS"]
    df["No. of Trades"] = bhav["NO_TRADES"]
    df["Total Turnover (Rs.)"] = bhav["NET_TURNOV"]
    df["Spread High-Low"] = bhav["HIGH"] - bhav["LOW"]
    df["Spread Close-Open"] = bhav["CLOSE"] - bhav["OPEN"]
    df = pd.merge(df, deliverableData, on='SC_CODE', how='left')
    return df

In [32]:
df = pd.DataFrame()
df = convertBhavToStock(df)
df

      SC_CODE  Deliverable Quantity  % Deli. Qty to Traded Qty
0      500002                  5123                      28.01
1      500003                  6694                      46.18
2      500008                  8571                      14.31
3      500009                 55411                      72.03
4      500010                 18373                      31.81
...       ...                   ...                        ...
3412   972715                     4                     100.00
3413   972719                     8                      66.67
3414   972770                    24                     100.00
3415   972773                    20                     100.00
3416   972885                     8                     100.00

[3417 rows x 3 columns]


,SC_CODE,SC_NAME,Open Price,High Price,Low Price,Close Price,WAP,No.of Shares,No. of Trades,Total Turnover (Rs.),Spread High-Low,Spread Close-Open,Deliverable Quantity,% Deli. Qty to Traded Qty
0,500002,ABB LTD.,1445.0,1458.2,1405.20,1421.00,1421.949475,18288,1421,26004612.0,53.00,-24.00,5123,28.01
1,500003,AEGIS LOGIS,295.0,310.8,295.00,302.45,300.800855,14497,828,4360710.0,15.80,7.45,6694,46.18
2,500008,AMAR RAJA BA,1000.0,1009.2,975.15,983.10,988.289333,59886,2183,59184695.0,34.05,-16.90,8571,14.31
3,500009,A.SARABHAI,21.2,22.5,20.65,21.20,21.451396,76928,286,1650213.0,1.85,0.00,55411,72.03
4,500010,HDFC,2735.0,2767.0,2735.00,2758.65,2752.947193,57757,3841,159001971.0,32.00,23.65,18373,31.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3412,972715,IFCI150212C,11055.0,11055.0,11055.00,11055.00,11055.000000,4,2,44220.0,0.00,0.00,4,100.00
3413,972719,IFCI310312A,10190.0,10220.0,10190.00,10220.00,10210.000000,12,3,122520.0,30.00,30.00,8,66.67
3414,972770,990IFCI22A,25150.0,25150.0,25150.00,25150.00,25150.000000,24,3,603600.0,0.00,0.00,24,100.00
3415,972773,990IFCI37D,22275.0,22275.0,22275.00,22275.00,22275.000000,20,1,445500.0,0.00,0.00,20,100.00
